In [1]:
import ROOT
from ROOT import gStyle
import numpy as np
import ctypes
import os
import pandas as pd

def biner(edges,bin_widths,histogram):
    if (len(edges)+1!=len(bin_widths)):
        print("Check edges and bin widths array sizes!")
        return
    
    bins=[]
    first_bin = histogram.GetXaxis().GetBinLowEdge(1)
    print(first_bin)
    last_bin = histogram.GetXaxis().GetBinUpEdge(histogram.GetNbinsX())
    print(last_bin)
    for i in range(0,len(edges)):
        n_spaces = int((edges[i] - first_bin)/bin_widths[i])
        bins = np.concatenate((bins,np.linspace(first_bin,edges[i],n_spaces,endpoint=False)))
        first_bin = edges[i]
        if edges[i]==edges[-1]:
            n_spaces = int((last_bin - edges[i])/bin_widths[i+1])
            bins = np.concatenate((bins,np.linspace(edges[i],last_bin,n_spaces,endpoint=False)))
            bins = np.concatenate((bins,[last_bin]))
    return bins

def normalization(hist_list,norm_bin):
    for hist in hist_list:
        for i in range(1,hist.GetNbinsX()+1):
            value=hist.GetBinContent(i)
            error=hist.GetBinError(i)
            sf=hist.GetBinWidth(i)/norm_bin
            hist.SetBinContent(i,value/sf)
            hist.SetBinError(i,error/sf)

def Plot(data,signal,background,histograms,watermark,average=False,after_fit=False,final_state="Z#rightarrow #mu#mu"):
    samples = data.copy()
    samples.update(background)
    samples.update(signal)

    for i in histograms:
        print(i)
        for s in samples:
            file = ROOT.TFile.Open(samples[s][0],"READ")
            hist = file.Get(i)
            hist.SetDirectory(0)
            samples[s].append
            samples[s][2]=hist # add histogram (TH1F) to list of samples
            file.Close()
        if average:
            watermark = "Average"
            if after_fit:
                watermark = "Average_AfterFit"
            
        ###### REBIN AND NORMALISE ######
        if len(histograms[i])>2:
            rebining=biner(histograms[i][0],histograms[i][1],samples["Data"][2])
            print(rebining)
            nb=len(rebining)-1
            for s in samples:
                samples[s][2]=samples[s][2].Rebin(nb,s,rebining)
            hist_list=[samples[s][2] for s in samples]
            normalization(hist_list,histograms[i][2])

        ###### SETTING THE COLOURS ######

        for s in samples:
            if s=="Data":
                samples[s][2].SetMarkerStyle(20)
                samples[s][2].SetMarkerSize(0.5)
                samples[s][2].SetLineColor(samples[s][1])
            else :
                #if s=="Signal" or s=="QCD Z" :
                    #samples[s][2].SetLineWidth(0)
                samples[s][2].SetFillColor(samples[s][1])
                samples[s][2].SetLineColor(samples[s][1]+1)

        #################### SCALING FACTORS FROM FIT ####################

        if after_fit:
            samples["Signal"][2].Scale(vbf_scale)
            samples["QCD Z"][2].Scale(qcd_scale)
                
        ####################### CREATING MC AND STACK HISTOGRAM ########################        
        
        hs = ROOT.THStack("hs","")
        mc = samples["Signal"][2].Clone()
        for s in samples:
            if s!="Data":
                hs.Add(samples[s][2])
                if s!="Signal":
                    mc.Add(samples[s][2],1)

        ############### DEFINING RATIOS ###############

        ratio = mc.Clone()
        ratio.Divide(samples["Data"][2])
    
        ratio_sg_mc=samples["Signal"][2].Clone()
        ratio_sg_mc.Divide(mc)


        ##### DRAWING TOP PAD, SETTING MARGINS #######
        
        gStyle.SetOptStat(1111111)
        gStyle.SetStatY(0.97);                
        gStyle.SetStatX(1.0);
        gStyle.SetStatW(0.12);                
        gStyle.SetStatH(0.12);

        canvas = ROOT.TCanvas("canvas2")
        canvas.cd()

        pad1 = ROOT . TPad (" pad1 "," pad1 " ,0 ,0.35 ,1 ,1)
        pad1.SetTopMargin(0.03)
        pad1.SetRightMargin(0.03)
        pad1.SetLeftMargin(0.08)
        pad1.SetBottomMargin(0.0)
        pad1.Draw ()
        pad1.cd ()

        ###### SETTING STATS BOX POSITION ######
        
        gStyle.SetStatY(0.95);                
        gStyle.SetStatX(0.96);
        gStyle.SetStatW(0.1);                
        gStyle.SetStatH(0.1);

        
        samples["Data"][2].Draw("pe same")
        hs.Draw("HIST same")
        samples["Data"][2].Draw("pe same")
        samples["Data"][2].Draw("sameaxis")
        
        pad1.SetLogy()
        
        s=samples["Data"][2].GetXaxis().GetBinLowEdge(1)
        e=samples["Data"][2].GetXaxis().GetBinUpEdge(samples["Data"][2].GetNbinsX())
        
        samples["Data"][2].GetYaxis().SetRangeUser(0.1 ,13*samples["Data"][2].GetBinContent(samples["Data"][2].GetMaximumBin()))
        samples["Data"][2].GetXaxis().SetRangeUser(s,e)
        hs.GetXaxis().SetRangeUser(s,e)
        if len(histograms[i])>2:
            samples["Data"][2].GetYaxis().SetTitle("Events/"+str(histograms[i][2])+" GeV")
        legend = ROOT . TLegend (0.45 ,0.80 ,0.85 ,0.95)
        for s in samples:
            legend.AddEntry(samples[s][2],s)
        legend.SetNColumns(3)
        gStyle.SetLegendBorderSize(0)
        legend . SetLineWidth (0)
        legend . Draw ()

        samples["Data"][2].SetTitle("")
        l=ROOT.TLatex()
        l.SetNDC ()
        l.DrawLatex(0.9,0.7,final_state)
        
        s=samples["Data"][2].GetXaxis().GetBinLowEdge(1)
        e=samples["Data"][2].GetXaxis().GetBinUpEdge(samples["Data"][2].GetNbinsX())
        
        max_ratio = ratio.GetMaximum()
        if max_ratio > 3:
            max_ratio = 3
        if max_ratio < 1.5:
            max_ratio = 1.5

        min_ratio = ratio.GetMinimum()
        if min_ratio > 0.7:
            min_ratio = 0.5
            
        canvas.cd()
        pad2 = ROOT . TPad (" pad2 "," pad2 " ,0 ,0.17 ,1 ,0.35)
        pad2.SetRightMargin(0.03)
        pad2.SetLeftMargin(0.08)
        pad2.SetTopMargin(0)
        pad2.SetBottomMargin(0.0)
        pad2.Draw ()
        pad2.cd ()
        ratio.SetFillColorAlpha(ROOT.kBlue,0.35)
        ratio.Draw ("E2")
        ratio.SetTitle("")
        ratio.SetStats(0)
        ratio . GetYaxis (). SetRangeUser (min_ratio ,max_ratio)
        ratio . GetXaxis (). SetRangeUser (s ,e)
        ratio . GetYaxis (). SetTitle ("MC/DATA")
        ratio . GetYaxis (). SetTitleSize (0.15)
        ratio . GetYaxis (). SetTitleOffset (0.25)
        ratio . GetXaxis (). SetTitleSize (0.09)
        ratio.GetXaxis().SetLabelSize(0.10)
        ratio.GetYaxis().SetLabelSize(0.08)
        ratio.SetMarkerStyle(8)
        ratio.SetMarkerSize(0.6)


        ###### SETTING ALL THE HORIZONTAL DASHED LINES #######

        line = ROOT . TLine (s ,1 ,e,1)
        line . SetLineColor ( ROOT . kBlack )
        line . SetLineWidth (2)

        separators = []
        resolution = 10 # In percentage
        j=0
        range_sep = []
        while j < (int(max_ratio)+1):
            step = resolution/100
            if j > min_ratio:
                range_sep.append(j)
            j = j + step

        for k in range (len(range_sep)):
            sep = ROOT.TLine(s ,range_sep[k],e,range_sep[k])
            sep.SetLineColor(ROOT.kBlack)
            sep.SetLineWidth(1)
            sep.SetLineStyle(2)
            separators.append(sep)

        line . Draw (" same ")   
        for separ in separators:
            separ.Draw("same")
        canvas.cd()
        pad3 = ROOT . TPad (" pad3","pad3" ,0.0 ,0.0 ,1 ,0.17)
        pad3.SetRightMargin(0.03)
        pad3.SetLeftMargin(0.08)
        pad3.SetTopMargin(0)
        pad3.SetBottomMargin(0.4)
        pad3.Draw ()
        pad3.cd ()
        ratio_sg_mc.SetStats(0)
        ratio_sg_mc . GetYaxis (). SetRangeUser (0.0 ,1.02)
        ratio_sg_mc . GetXaxis (). SetRangeUser (s ,e)
        ratio_sg_mc . GetYaxis (). SetTitle ("SIGNAL/MC")
        ratio_sg_mc . GetYaxis (). SetTitleSize (0.15)
        ratio_sg_mc . GetYaxis (). SetTitleOffset (0.25)
        ############ X AXIS TITLE #################
        axisTitle='hola'
        if len(histograms[i])>2:
            try :
                axisTitle=histograms[i][3]    
            except :
                pass
        else :
            try :
                axisTitle=histograms[i][0]    
            except :
                pass
        ##########################################
        ratio_sg_mc.SetXTitle(axisTitle)
        ratio_sg_mc.SetTitleSize(0.17,"X")
        ratio_sg_mc.SetTitleOffset(0.9,"X")
        ratio_sg_mc.GetYaxis().SetLabelSize(0.09)
        ratio_sg_mc.GetXaxis().SetLabelSize(0.16)
        ratio_sg_mc.SetMarkerStyle(8)
        ratio_sg_mc.SetMarkerSize(0.6)

        ratio_sg_mc.Draw ("hist p")
        ###### SETTING ALL THE HORIZONTAL DASHED LINES #######

        line11 = ROOT . TLine (s ,0.80 ,e,0.80)
        line11 . SetLineColor ( ROOT . kBlack )
        line11 . SetLineWidth (1)
        line11 . SetLineStyle(2)
        line12 = ROOT . TLine (s ,0.60 ,e,0.60)
        line12 . SetLineColor ( ROOT . kBlack )
        line12 . SetLineWidth (1)
        line12 . SetLineStyle(2)
        line13 = ROOT . TLine (s ,0.40 ,e,0.40)
        line13 . SetLineColor ( ROOT . kBlack )
        line13 . SetLineWidth (1)
        line13 . SetLineStyle(2)
        line14 = ROOT . TLine (s ,0.20 ,e,0.20)
        line14 . SetLineColor ( ROOT . kBlack )
        line14 . SetLineWidth (1)
        line14 . SetLineStyle(2)
        sline = ROOT . TLine (s ,1 ,e,1)
        sline . SetLineColor ( ROOT . kBlack )
        sline . SetLineWidth (2)
        sline.Draw("same")
        line11 . Draw (" same ")
        line12 . Draw (" same ")
        line13 . Draw (" same ")
        line14 . Draw (" same ")
        
        file_name = i+"_"+watermark+".pdf"
        canvas.Update()
        canvas.Print(file_name)

Welcome to JupyROOT 6.24/06
